In [1]:
import torchaudio
import torch
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from ast import literal_eval

# from torchsummary import summary

# dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [2]:
#Use to reload a module
import importlib
import helpers

importlib.reload(helpers)

from helpers.AudioVisionDataset import AudioVisionDataset
from helpers.ModelLayout import AudioVisionModel

In [3]:
features_file = "../DataAnalysis/8KFeatures.csv"

In [4]:
# featuresdf = pd.read_csv(features_file)
# temp = literal_eval(featuresdf['feature'][0])
# print(temp)
# print(len(temp))

In [5]:
av_dataset = AudioVisionDataset(features_file, dtype)

In [6]:
train_size = int(0.667 * len(av_dataset))
test_size = len(av_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(av_dataset, [train_size, test_size])

In [7]:
model = AudioVisionModel()
model.cuda()

AudioVisionModel(
  (seq1): Sequential()
  (conv1): Conv2d(1, 14, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(14, 28, kernel_size=(2, 2), stride=(1, 1))
  (conv3): Conv2d(28, 42, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(42, 56, kernel_size=(2, 2), stride=(1, 1))
  (drop1): Dropout(p=0.2, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (dense1): Linear(in_features=1456, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=10, bias=True)
)

In [8]:
import torch.utils.data as data
train_data_loader = data.DataLoader(train_dataset, shuffle = True)
test_data_loader = data.DataLoader(test_dataset, shuffle = True)

In [9]:
# summary(model, (1,14, 109))

In [10]:
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.7)

In [15]:
import time

def trainNet(net, n_epochs):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", 174)
    print("epochs=", n_epochs)
    print("learning_rate=", 0.001)
    print("=" * 30)
    
    n_batches = len(train_data_loader)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        count = 0
        
        for i, data in enumerate(train_data_loader, 0):
            count+= 1
            
            #Get inputs
            inputs, labels = data
            inputs = inputs.unsqueeze(0)
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
#             print(labels.dtype)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels[0])
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in test_data_loader:
            
            count += 1
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs.unsqueeze(0)).cuda(), Variable(labels).cuda()
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels[0])
            total_val_loss += val_loss_size.data
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(test_data_loader)))
        print(count)
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [16]:
trainNet(model, 5)

===== HYPERPARAMETERS =====
batch_size= 174
epochs= 5
learning_rate= 0.001
Epoch 1, 10% 	 train_loss: 2.26 took: 4.62s
Epoch 1, 20% 	 train_loss: 2.25 took: 4.59s
Epoch 1, 30% 	 train_loss: 2.28 took: 4.68s
Epoch 1, 40% 	 train_loss: 2.25 took: 4.76s
Epoch 1, 50% 	 train_loss: 2.28 took: 4.59s
Epoch 1, 60% 	 train_loss: 2.27 took: 4.65s
Epoch 1, 70% 	 train_loss: 2.27 took: 4.67s
Epoch 1, 80% 	 train_loss: 2.27 took: 4.65s
Epoch 1, 90% 	 train_loss: 2.26 took: 4.64s
Validation loss = 2.26
8732
Epoch 2, 10% 	 train_loss: 2.27 took: 4.62s
Epoch 2, 20% 	 train_loss: 2.27 took: 4.58s


KeyboardInterrupt: 

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)